In [35]:
import json
import pandas as pd
import psycopg2


In [36]:
# Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [37]:
# Reading file with connection secret information
with open ('data/secret.json', 'r') as f:
    secret = json.load(f)

In [38]:
# Connection to sql
connection = psycopg2.connect(
    dbname = secret['DBNAME'],
    user = secret['USER'],
    password = secret['PASSWORD'],
    host = secret['HOST'],
    port = secret['PORT']
)

In [39]:
# Finding quantity of vacancies
query_3_1 = f'''select count(*)
                from public.vacancies'''
df = pd.read_sql_query(query_3_1, connection)
total_vacancies = df["count"][0]
print(f'Количество вакансий: {df["count"][0]}')

Количество вакансий: 49197


In [40]:
# Finding quantity of employers
query_3_2 = f'''select count(*)
                from public.employers'''
df = pd.read_sql_query(query_3_2, connection)
print(f'Количество работодателей: {df["count"][0]}')

Количество работодателей: 23501


In [41]:
# Finding quantity of areas
query_3_3 = f'''select count(*)
                from public.areas'''
df = pd.read_sql_query(query_3_3, connection)
print(f'Количество регионов: {df["count"][0]}')

Количество регионов: 1362


In [42]:
# Finding quantity of indastries
query_3_4 = f'''select count(*)
                from public.industries'''
df = pd.read_sql_query(query_3_4, connection)
print(f'Количество сфер деятельности: {df["count"][0]}')

Количество сфер деятельности: 294


Исходя из проведенного предварительного анализа данных можно сделать следующие выводы:

В базе данных содержится информация о 49197 вакансиях от 23501 работодателей, размещенных в 1362 регионах и принадлежащих к 294 сферам деятельности.

In [43]:
# Finding top 5 cities by quantity of vacancies
query_4_1 = f'''select
                    a.name,
                    count(v.id) as vacancies_count
                from public.vacancies as v
                    join public.areas as a on a.id = v.area_id
                group by a.name
                order by vacancies_count desc
                limit 5'''
df = pd.read_sql_query(query_4_1, connection)
display(df)

,name,vacancies_count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


In [44]:
# Finding quantity of vacancies with at least one salary field filled 
query_4_2 = f'''select count(*)
                from public.vacancies
                where salary_from is not null OR salary_to is not null'''
df = pd.read_sql_query(query_4_2, connection)
print(f'Количество вакансий у которых заполнено хотябы одно поле З/П: {df["count"][0]}')

Количество вакансий у которых заполнено хотябы одно поле З/П: 24073


In [45]:
# Finding mean lower and upper bound of salary
query_4_3 = f'''select
                    avg(salary_from) AS lower_bound,
                    avg(salary_to) AS upper_bound
                from public.vacancies'''
df = pd.read_sql_query(query_4_3, connection)
print(f'Нижняя граница: {round(df["lower_bound"][0])}')
print(f'Верхняя граница: {round(df["upper_bound"][0])}')

Нижняя граница: 71065
Верхняя граница: 110537


In [46]:
# Finding vacancies quantity by type of schedule
query_4_4 = f'''select
                    schedule,
                    count(8)
                from public.vacancies
                group by schedule
                order by count desc'''
df = pd.read_sql_query(query_4_4, connection)
display(df)

,schedule,count
0,Полный день,36424
1,Удаленная работа,9311
2,Гибкий график,2039
3,Сменный график,1054
4,Вахтовый метод,369


In [47]:
# Finding vacancies quantity by experience
query_4_5 =f'''select
                    experience,
                    count(id)
                from public.vacancies
                where experience is not null
                group by experience
                order by count'''
df = pd.read_sql_query(query_4_5, connection)
display(df)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152



По результатам детального анализа вакансий можно сделать следующие выводы:

+ Самое большое количество вакансий находится в Москве (5333), Санкт-Петербурге (2851), Минске (2112), Новосибирске (2006) и Алматы (1892).
+ Заполнено поле З/П хотя бы у одного из двух полей верхней или нижней границы в 24073 вакансиях.
+ Средняя нижняя граница зарплаты составляет 71065 рублей, а верхняя граница - 110537 рублей.
+ Больше всего вакансий с графиком - Полный рабочий день(36424), на втором месте по количеству вакансий оказалась удаленная работа (9311 вакансий).
+ Большинство вакансий требуют опыта работы от 1 года до 3 лет (26152 вакансии), за ним идут вакансии требующие от 3 до 6 лет опыта (14511 вакансий), Более 6 лет (1337 вакансий) и Нет опыта (7197 вакансий).

In [48]:
# Finding top 5 employers
query_5_1 = f'''select
                    e.name,
                    count(v.id)
                from public.vacancies as v
                    join public.employers as e on e.id = v.employer_id
                group by e.name
                order by count(v.id) desc
                limit 5'''
df = pd.read_sql_query(query_5_1, connection)
display(df)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [49]:
# Finding top 5 areas by employers qantity with null vacancies
query_5_2 = f'''
SELECT 
    DISTINCT a.name,
    COUNT(DISTINCT e.id) AS employers_count,
    vac.vacancies_count
FROM public.areas AS a
LEFT JOIN public.employers AS e ON e.area = a.id
FULL JOIN (
    SELECT 
        a.name,
        COUNT(v.id) AS vacancies_count
    FROM public.areas AS a
    JOIN public.vacancies AS v ON v.area_id = a.id
    GROUP BY a.name
) AS vac ON a.name = vac.name
GROUP BY a.name, vac.vacancies_count
HAVING COUNT(vac.vacancies_count) = 0
ORDER BY employers_count DESC
LIMIT 5'''
df = pd.read_sql_query(query_5_2, connection)
display(df)

In [ ]:
# Counting quantity recruiting regions for each company
query_5_3 = f'''
SELECT 
    e.name,
    count(distinct v.area_id) as region_count
FROM public.vacancies AS v
    JOIN public.employers AS e ON e.id = v.employer_id
GROUP BY e.name
ORDER BY count(v.area_id) DESC
'''
df = pd.read_sql_query(query_5_3, connection)
display(df)

,name,region_count
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
...,...,...
14761,ПСБ Инновации и Инвестиции,1
14762,ПСЗ Янтарь,1
14763,ПСК,1
14764,Псковский завод механических приводов,1


In [ ]:
# Finding quantity of employers with null in industry type
query_5_4 = '''
SELECT count(*)
FROM public.employers AS e
    LEFT JOIN public.employers_industries AS ei ON ei.employer_id = e.id
WHERE industry_id is null'''
df = pd.read_sql_query(query_5_4, connection)
print(f'Количество работадателей у которых не указана сфера деятельности: {df["count"][0]}')

Количество работадателей у которых не указана сфера деятельности: 8419


In [ ]:
# Finding employer name with four industry types in third place in the alphabetical list                  
query_5_5 = '''select
                    e.name
                from public.employers as e
                    join public.employers_industries as ei on e.id = ei.employer_id
                    group by e.name
                    having count(ei.industry_id) = 4
                    order by name
                    OFFSET 2
                    LIMIT 1
                    '''
df = pd.read_sql_query(query_5_5, connection)
print(f'Название компании с 4 меня сферами деятельности на третьем месте в алфавитном списке: {df["name"][0]}')

Название компании с 4 меня сферами деятельности на третьем месте в алфавитном списке: 2ГИС


In [ ]:
# Finding quantity of software developers company
query_5_6 = '''
SELECT count(ei.employer_id)
FROM public.employers_industries AS ei
    JOIN public.industries AS i ON i.id = ei.industry_id
WHERE i.name = 'Разработка программного обеспечения'
GROUP BY i.name
'''
df = pd.read_sql_query(query_5_6, connection)
print(f'Компаний разработчиков ПО: {df["count"][0]}')

Компаний разработчиков ПО: 3553


In [ ]:
# Geting million_cities tuple from wikipedia
million_cities_link = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
million_cities = tuple(pd.read_html(million_cities_link)[1]['Город'])
#  = tuple(table['Город'])

In [ ]:
# Finding quantity of Yandex vacancies in million cities
query_5_7 = f'''
(SELECT 
    a.name,
    count(v.id)
FROM public.vacancies AS v
    JOIN public.areas AS a ON a.id = v.area_id
    JOIN public.employers AS e ON e.id = v.employer_id
WHERE e.name = 'Яндекс' AND a.name IN {million_cities}
GROUP BY a.name
ORDER BY count(v.id)DESC)

UNION ALL
SELECT
    'Total',
    SUM(cnt)
FROM (
    SELECT 
        count(v.id) AS cnt
    FROM 
        public.vacancies AS v
        JOIN public.areas AS a ON a.id = v.area_id
        JOIN public.employers AS e ON e.id = v.employer_id
    WHERE 
        e.name = 'Яндекс' AND 
        a.name IN {million_cities}
    GROUP BY a.name
) AS t

'''
df = pd.read_sql_query(query_5_7, connection)
display(df)

,name,count
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


В результате анализа работадателей были сделаны следующие выводы:

+ Топ-5 работодателей по количеству вакансий: Яндекс (1933), Ростелеком (491), Тинькофф (444), СБЕР (428) и Газпром нефть (331).
+ Топ-5 регионов по количеству работодателей с нулевым количеством вакансий: Россия (410), Казахстан (207), Московская область (75), Краснодарский край (19) и Беларусь (18).
+ Топ 5 количества регионов, в которых каждая компания ищет сотрудников: Яндекс (181), Ростелеком (152), Тинькофф (43), СБЕР (24) и Газпром нефть (24).
+ Количество работодателей, у которых не указана сфера деятельности: 8419.
+ Компания с 4 сферами деятельности на третьем месте в алфавитном списке: 2ГИС.
+ Количество компаний, занимающихся разработкой программного обеспечения: 3553.
+ Компания Яндекс разместила 485 вакансий в городах миллионниках, больше всего в Москве(54), Санкт-Петербурге(42) и Екатеринбурге

In [ ]:
# Finding quntity of vacancies relevant to data
query_6_1 = '''
SELECT count(*)
FROM public.vacancies
WHERE LOWER(name) LIKE '%data%' OR LOWER(name) LIKE '%данн%'
'''
df = pd.read_sql_query(query_6_1, connection)
print(f'Колличество вакансий имеющих отношение к данным: {df["count"][0]}')
data_vacancies = df["count"][0]

Колличество вакансий имеющих отношение к данным: 1771


In [ ]:
# Finding vacancies quantity for beginer data scientist
query_6_2 = '''
SELECT count (*)
FROM public.vacancies
WHERE 
    (name ILIKE '%data scientist%' OR 
     name ILIKE '%data science%' OR 
     name ILIKE '%исследователь данных%' OR 
     (name LIKE '%ML%' AND name NOT LIKE '%HTML%') OR 
     name ILIKE '%machine learning%' OR 
     name ILIKE '%машинн%обучен%') 
    AND 
    (name ILIKE '%junior%' OR 
     experience = 'Нет опыта' OR 
     employment = 'Стажировка')
'''
df = pd.read_sql_query(query_6_2, connection)
print(f'Колличество вакансий для начинающего дата-сайентиста: {df["count"][0]}')
junior_ds_vac = df["count"][0]

Колличество вакансий для начинающего дата-сайентиста: 51


In [ ]:
# Finding DS vacancies quantity with SQL and Postgre SQL key skills
query_6_3 = '''
SELECT count(*)
FROM public.vacancies
WHERE 
    (name ILIKE '%data scientist%' OR 
     name ILIKE '%data science%' OR 
     name ILIKE '%исследователь данных%' OR 
     (name LIKE '%ML%' AND name NOT LIKE '%HTML%') OR 
     name ILIKE '%machine learning%' OR 
     name ILIKE '%машинн%обучен%') 
    AND 
    (key_skills ILIKE '%sql%' OR 
     key_skills ILIKE 'postgres')
'''
df = pd.read_sql_query(query_6_3, connection)
print(f'Колличество вакансий для дата-сайентиста с ключевыми навыками SQL или postgres: {df["count"][0]}')

Колличество вакансий для дата-сайентиста с ключевыми навыками SQL или postgres: 201


In [ ]:
# # Finding DS vacancies quantity with Python key skill
query_6_4 = '''
SELECT count(*)
FROM public.vacancies
WHERE 
    (name ILIKE '%data scientist%' OR 
     name ILIKE '%data science%' OR 
     name ILIKE '%исследователь данных%' OR 
     (name LIKE '%ML%' AND name NOT LIKE '%HTML%') OR 
     name ILIKE '%machine learning%' OR 
     name ILIKE '%машинн%обучен%') 
    AND 
    key_skills ILIKE '%python%'
'''
df = pd.read_sql_query(query_6_4, connection)
print(f'Колличество вакансий для дата-сайентиста с ключевым навыком Python: {df["count"][0]}')

Колличество вакансий для дата-сайентиста с ключевым навыком Python: 351


In [ ]:
# Finding mean key skills quantity for DS vacancies
query_6_5 = '''
SELECT ROUND(AVG(LENGTH(key_skills) - LENGTH(replace(key_skills, CHR(9), '')) + 1),2) as avg
FROM public.vacancies
WHERE 
    (name ILIKE '%data scientist%' OR 
     name ILIKE '%data science%' OR 
     name ILIKE '%исследователь данных%' OR 
     (name LIKE '%ML%' AND name NOT LIKE '%HTML%') OR 
     name ILIKE '%machine learning%' OR 
     name ILIKE '%машинн%обучен%') 
    AND 
    key_skills IS NOT NULL
'''
df = pd.read_sql_query(query_6_5, connection)
print(f'Среднее количество ключевых навыков для DS вакансий: {df["avg"][0]}')

Среднее количество ключевых навыков для DS вакансий: 6.41


In [ ]:
# Finding mean salary dor DS vacancies
query_6_6 = '''
SELECT experience,
       ROUND(AVG(CASE
             WHEN salary_to IS NOT NULL AND salary_from IS NOT NULL THEN (salary_to + salary_from)/2
             WHEN salary_to IS NULL THEN salary_from
             ELSE salary_to
           END)) as avg_salary
FROM public.vacancies
WHERE 
    (name ILIKE '%data scientist%' OR 
     name ILIKE '%data science%' OR 
     name ILIKE '%исследователь данных%' OR 
     (name LIKE '%ML%' AND name NOT LIKE '%HTML%') OR 
     name ILIKE '%machine learning%' OR 
     name ILIKE '%машинн%обучен%') 
    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
'''
ds_mean_salary_df = pd.read_sql_query(query_6_6, connection)
display(ds_mean_salary_df)

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0



В результате предметного анализа вакансий в базе данных были получены следующие выводы:

+ В базе данных имеется 1771 вакансия, имеющая отношение к данным.
+ Для начинающих дата-сайентистов в базе данных имеется всего 51 вакансия.
+ Для дата-сайентистов с ключевыми навыками SQL или Postgres имеется 201 вакансия.
+ Для дата-сайентистов с ключевым навыком Python имеется 351 вакансия.
+ Среднее количество ключевых навыков для вакансий в сфере Data Science составляет 6.41.
+ Средняя зарплата для вакансий в сфере Data Science зависит от опыта работы
    1. Без опыта работы: 74643 руб
    2. С опытом от года до 3х лет 139675 руб
    3. С опытом работы от 3х до 6 лет 243115 руб

In [ ]:
# Finding top 5 employers for DS vacancies
query_7_1 = '''
SELECT 
    e.name AS employer_name,
    count(v.id)
    
FROM public.vacancies AS v
    JOIN public.employers AS e ON v.employer_id = e.id
WHERE 
    (v.name ILIKE '%data scientist%' OR 
     v.name ILIKE '%data science%' OR 
     v.name ILIKE '%исследователь данных%' OR 
     (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR 
     v.name ILIKE '%machine learning%' OR 
     v.name ILIKE '%машинн%обучен%')
GROUP BY e.name
ORDER BY count(v.id) DESC
LIMIT 5
'''
df = pd.read_sql_query(query_7_1, connection)
display(df)

,employer_name,count
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11


In [ ]:
# Finding DS vacancies quantity by industries type
query_7_2 = '''
SELECT 
    i.name AS industry_name,
    count(v.id)
    
FROM public.vacancies AS v
    JOIN public.employers_industries AS ei ON v.employer_id = ei.employer_id
    JOIN public.industries AS i ON ei.industry_id = i.id
WHERE 
    (v.name ILIKE '%data scientist%' OR 
     v.name ILIKE '%data science%' OR 
     v.name ILIKE '%исследователь данных%' OR 
     (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR 
     v.name ILIKE '%machine learning%' OR 
     v.name ILIKE '%машинн%обучен%')
GROUP BY i.name
ORDER BY count(v.id) DESC
LIMIT 5
'''
df = pd.read_sql_query(query_7_2, connection)
display(df)

,industry_name,count
0,Разработка программного обеспечения,148
1,"Системная интеграция, автоматизации технологи...",139
2,Банк,92
3,"Интернет-компания (поисковики, платежные систе...",79
4,Мобильная связь,23


In [ ]:
# Finding DS vacancies quantity by employment type
query_7_3 = '''
SELECT 
    count(v.id)
    
FROM public.vacancies AS v
WHERE 
    (v.name ILIKE '%data scientist%' OR 
     v.name ILIKE '%data science%' OR 
     v.name ILIKE '%исследователь данных%' OR 
     (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR 
     v.name ILIKE '%machine learning%' OR 
     v.name ILIKE '%машинн%обучен%')
ORDER BY count(v.id) DESC
'''
df = pd.read_sql_query(query_7_3, connection)
print(f'Количество вакансий для дата-сайентистов: {df["count"][0]}')
ds_vac_quantity = df["count"][0]

Количество вакансий для дата-сайентистов: 480


In [ ]:
# Finding DS vacancies quantity by employment type
query_7_4 = '''
SELECT 
    v.employment AS employmet_type,
    count(v.id)
    
FROM public.vacancies AS v
WHERE 
    (v.name ILIKE '%data scientist%' OR 
     v.name ILIKE '%data science%' OR 
     v.name ILIKE '%исследователь данных%' OR 
     (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR 
     v.name ILIKE '%machine learning%' OR 
     v.name ILIKE '%машинн%обучен%')
GROUP BY v.employment
ORDER BY count(v.id) DESC
'''
df = pd.read_sql_query(query_7_4, connection)
display(df)

,employmet_type,count
0,Полная занятость,466
1,Частичная занятость,8
2,Стажировка,4
3,Проектная работа,2


In [ ]:
# Finding mean salry of all vacancies for compare with Data science vacancies
query_7_5 = '''
SELECT experience,
       ROUND(AVG(CASE
             WHEN salary_to IS NOT NULL AND salary_from IS NOT NULL THEN (salary_to + salary_from)/2
             WHEN salary_to IS NULL THEN salary_from
             ELSE salary_to
           END)) as avg_salary
FROM public.vacancies
WHERE 
  (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
ORDER BY experience
OFFSET 1
'''
mean_salary_df = pd.read_sql_query(query_7_5, connection)
salary_merged_df = pd.merge(left=mean_salary_df, right=ds_mean_salary_df, on='experience', suffixes=('_all', '_ds'))
salary_merged_df['difference%'] = round((salary_merged_df['avg_salary_ds']/salary_merged_df['avg_salary_all'] * 100) - 100)
display(salary_merged_df)


,experience,avg_salary_all,avg_salary_ds,difference%
0,Нет опыта,40472.0,74643.0,84.0
1,От 1 года до 3 лет,76541.0,139675.0,82.0
2,От 3 до 6 лет,133211.0,243115.0,83.0


In [ ]:
# Counting percents for conclusions
print(f'Процент от общего количества вакансий связанных с данными: {round(data_vacancies/total_vacancies * 100,1)}')
print(f'Процент от общего количества DataScience вакансий: {round(ds_vac_quantity/total_vacancies * 100,1)}')
mean_salary_diff = salary_merged_df['difference%'].mean()
print(f'В среднем зарплата дата-сайентистов выше рыночной на: {mean_salary_diff}')

Процент от общего количества вакансий связанных с данными: 3.6
Процент от общего количества DataScience вакансий: 1.0
В среднем зарплата дата-сайентистов выше рыночной на: 83.0


In [ ]:
connection.close()

В результате анализа данных от компании Head Hunter можно сделать следующие выводы:
+ В базе данных содержится информация о 49197 вакансиях от 23501 работодателей, размещенных в 1362 регионах и принадлежащих к 294 сферам деятельности
+ Больше всего вакансий размещено в Москве и Санкт-Петербурге
+ Большинство вакансия требуют от 1 года до 3х лет опыта работы
+ Средняя нижняя граница зарплаты составляет 71065 рублей, а верхняя граница - 110537 рублей
+ Больше всего вакансий с графиком - Полный рабочий день(36424), на втором месте по количеству вакансий оказалась удаленная работа (9311 вакансий).
+ Большинство вакансий требуют опыта работы от 1 года до 3 лет (26152 вакансии), за ним идут вакансии требующие от 3 до 6 лет опыта (14511 вакансий), Более 6 лет (1337 вакансий) и Нет опыта (7197 вакансий)
+ Топ-5 работодателей по количеству вакансий: Яндекс (1933), Ростелеком (491), Тинькофф (444), СБЕР (428) и Газпром нефть (331)
+ Топ 5 количества регионов, в которых каждая компания ищет сотрудников: Яндекс (181), Ростелеком (152), Тинькофф (43), СБЕР (24) и Газпром нефть (24).
+ Количество работодателей, у которых не указана сфера деятельности: 8419.
+ Количество компаний, занимающихся разработкой программного обеспечения: 3553.
+ Компания Яндекс разместила 485 вакансий в городах миллионниках, больше всего в Москве(54), Санкт-Петербурге(42) и Екатеринбурге
+ Из 49197 вакансий базы данных Head Hunter 1771 имеют отношение к данным что составляет 3.6 процента из них 51 вакансия подходит для начинающего дата-сайентиста
+ Количество дата-сайнс вакансий - 480 что состовляет 1 процент от общего количества
+ Большинство вакансий для дата - сайентистов подразумевают полный рабочий день, 
+ Топ 5 работадателй по количеству вакансий для дата-сайентистов СБЕР(37), Bell Integrator(25), Банк ВТБ (ПАО)(18), VK (15), Positive Technologies (11)
+ Больше всего вакансий для дата-сайентистов представленно в следующих сферах:
    1. Разработка программного обеспечения - 148 вакансий
    2. Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг - 139 вакансий
    3. Банк - 92 вакансии
    4. Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее) - 79 вакансий
    5. Мобильная связь
+ В среднем зарплата дата-сайентистов выше средней по рынку для аналогичного опыта работы на 83%